- Carla I. Gonzalez
- I320D

# Assignment 3: Logistic Regression and SVMs

#### On this assignment we will use data recorded for  Mt Rainer National park to predict if the user should or should not climb that day.

#### Here one can read columns descriptions: https://www.kaggle.com/datasets/codersree/mount-rainier-weather-and-climbing-data

In [84]:
# Load and process the data, import packages 
import sys
!conda install --yes --prefix {sys.prefix} pandas

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Solving environment: done

# All requested packages already installed.



In [85]:
#clean data, drop duplicates
import pandas as pd
import numpy
from sklearn.preprocessing import OneHotEncoder
import datetime

df = pd.read_csv("MtRainier_data.csv")
df = df.drop_duplicates()

df = df.dropna()
print (f"Shape of data {df.shape}")
df.head()

Shape of data (1895, 10)


,Unnamed: 0,Date,Route,Succeeded,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,0,11/27/2015,Disappointment Cleaver,0,13.643750,26.321667,19.715000,27.839583,68.004167,88.496250
1,1,11/21/2015,Disappointment Cleaver,0,13.749583,31.300000,21.690708,2.245833,117.549667,93.660417
2,2,10/15/2015,Disappointment Cleaver,0,13.461250,46.447917,27.211250,17.163625,259.121375,138.387000
3,3,10/13/2015,Little Tahoma,0,13.532083,40.979583,28.335708,19.591167,279.779167,176.382667
4,4,10/9/2015,Disappointment Cleaver,0,13.216250,38.260417,74.329167,65.138333,264.687500,27.791292


In [86]:
#Feature engineering 
#create a df of features and one for the label
features_df = df[["Unnamed: 0", "Date", "Route", "Battery Voltage AVG", "Temperature AVG", "Relative Humidity AVG", "Wind Speed Daily AVG", "Solare Radiation AVG"]]
labels_df = df[["Succeeded"]]

features_df.head()

,Unnamed: 0,Date,Route,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Solare Radiation AVG
0,0,11/27/2015,Disappointment Cleaver,13.643750,26.321667,19.715000,27.839583,88.496250
1,1,11/21/2015,Disappointment Cleaver,13.749583,31.300000,21.690708,2.245833,93.660417
2,2,10/15/2015,Disappointment Cleaver,13.461250,46.447917,27.211250,17.163625,138.387000
3,3,10/13/2015,Little Tahoma,13.532083,40.979583,28.335708,19.591167,176.382667
4,4,10/9/2015,Disappointment Cleaver,13.216250,38.260417,74.329167,65.138333,27.791292


In [87]:
labels_df.head()

,Succeeded
0,0
1,0
2,0
3,0
4,0


We have two categorical values, date and route. 
So we will apply encoding to make them numerical values:

First I will transfrom route with 1 hot enconding, and then I will extract month from datetime:

In [129]:
# Transform categorical features into 1-hot
route_to_list = features_df["Route"].to_list()
route_to_list_of_lists = []

for route in route_to_list:
    route_to_list_of_lists.append([route])
    
route_encoder = OneHotEncoder()
route_encoder.fit(route_to_list_of_lists)

print(f"Unique vocabulary items {len(route_encoder.categories_[0])}\n")

route_transformed = route_encoder.transform(route_to_list_of_lists)
route_transformed = route_transformed.toarray()
route_transformed_df = pd.DataFrame(route_transformed)

features_df.reset_index(drop=True, inplace=True)
route_transformed_df.reset_index(drop=True, inplace=True)

features_transformed_df = pd.concat([features_df,route_transformed_df], axis=1)
features_transformed_df = features_transformed_df.drop(columns=["Route"], axis=1)


#date to month
features_transformed_df['month'] = pd.DatetimeIndex(features_transformed_df['Date']).month
features_transformed_df = features_transformed_df.drop(columns=["Date"], axis=1)
features_transformed_df.head()

OneHotEncoder()

Unique vocabulary items 22



,Unnamed: 0,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Solare Radiation AVG,0,1,2,3,...,13,14,15,16,17,18,19,20,21,month
0,0,13.643750,26.321667,19.715000,27.839583,88.496250,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
1,1,13.749583,31.300000,21.690708,2.245833,93.660417,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
2,2,13.461250,46.447917,27.211250,17.163625,138.387000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,3,13.532083,40.979583,28.335708,19.591167,176.382667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,4,13.216250,38.260417,74.329167,65.138333,27.791292,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


Scale using standard scaling:

In [130]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

features_transformed_df[["Unnamed: 0", "Battery Voltage AVG", "Temperature AVG", "Relative Humidity AVG", "Wind Speed Daily AVG", "Solare Radiation AVG", "month"]] = scaler.fit_transform(features_transformed_df[["Unnamed: 0", "Battery Voltage AVG", "Temperature AVG", "Relative Humidity AVG", "Wind Speed Daily AVG", "Solare Radiation AVG", "month"]])
features_transformed_df.head()

,Unnamed: 0,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Solare Radiation AVG,0,1,2,3,...,13,14,15,16,17,18,19,20,21,month
0,-1.731137,2.003522,-1.580891,-1.269311,1.895222,-1.567664,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.129083
1,-1.729309,3.506158,-1.033951,-1.180109,-0.902775,-1.520897,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.129083
2,-1.727481,-0.587638,0.630261,-0.930861,0.728090,-1.115850,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.417416
3,-1.725653,0.418063,0.029488,-0.880092,0.993477,-0.771758,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.417416
4,-1.723825,-4.066182,-0.269251,1.196481,5.972851,-2.117412,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.417416


Spli data:

In [138]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

features = features_transformed_df.to_numpy()
labels = labels_df.to_numpy()
_x, x_test, _y, y_test = train_test_split(features, labels, test_size=0.10, random_state=42)

# set k = 5
k = 5

kfold_spliiter = KFold(n_splits=k)

folds_data = [] 

fold = 1
for train_index, validation_index in kfold_spliiter.split(_x):
    x_train , x_valid = _x[train_index,:],_x[validation_index,:]
    y_train , y_valid = _y[train_index,:] , _y[validation_index,:]
    print (f"Fold {fold} training data shape = {(x_train.shape,y_train.shape)}")
    print (f"Fold {fold} validation data shape = {(x_valid.shape,y_valid.shape)}")
    fold+=1
    folds_data.append((x_train,y_train,x_valid,y_valid))

Fold 1 training data shape = ((1364, 29), (1364, 1))
Fold 1 validation data shape = ((341, 29), (341, 1))
Fold 2 training data shape = ((1364, 29), (1364, 1))
Fold 2 validation data shape = ((341, 29), (341, 1))
Fold 3 training data shape = ((1364, 29), (1364, 1))
Fold 3 validation data shape = ((341, 29), (341, 1))
Fold 4 training data shape = ((1364, 29), (1364, 1))
Fold 4 validation data shape = ((341, 29), (341, 1))
Fold 5 training data shape = ((1364, 29), (1364, 1))
Fold 5 validation data shape = ((341, 29), (341, 1))


Create models: 

In [139]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

lr_vanilla = LogisticRegression(penalty= "none")
lr_L2 = LogisticRegression(penalty="l2")

svm_linear = SVC(kernel="linear")
svm_poly = SVC(kernel="poly",degree=2)


all_models = {"lr_vanilla":lr_vanilla, 
              "lr_L2":lr_L2,
              "svm_linear":svm_linear,
              "svm_poly":svm_poly}

In [137]:
best_validation_accuracy = 0
best_model_name = ""
best_model = None

for model_name in all_models.keys():
    
    print (f"Evaluating {model_name} ...")
    model = all_models[model_name]
    
    train_acc_for_all_folds = []
    valid_acc_for_all_folds = []
    
    for i, fold in enumerate(folds_data):
        x_train, y_train, x_valid, y_valid = fold

        _ = model.fit(x_train,y_train.flatten())

        y_pred_train = model.predict(x_train)
        
        y_pred_valid = model.predict(x_valid)
        
        train_acc = accuracy_score(y_pred_train , y_train.flatten())
        
        train_acc_for_all_folds.append(train_acc)
        
        valid_acc = accuracy_score(y_pred_valid , y_valid.flatten())

        valid_acc_for_all_folds.append(valid_acc)
    
    avg_training_acc = sum(train_acc_for_all_folds)/k
    
    print (f"Average training accuracy for model {model_name} = {avg_training_acc}")
    
    avg_validation_acc = sum(valid_acc_for_all_folds)/k
    
    print (f"Average validation accuracy for model {model_name} = {avg_validation_acc}")
    
    if avg_validation_acc > best_validation_accuracy:
        best_validation_accuracy = avg_validation_acc
        best_model_name = model_name
        best_model = model
    print (f"-----------------------------------")

print (f"Best model for the task is {best_model_name} which offers the validation accuracy of {best_validation_accuracy}")

Evaluating lr_vanilla ...
Average training accuracy for model lr_vanilla = 0.6356304985337243
Average validation accuracy for model lr_vanilla = 0.6217008797653959
-----------------------------------
Evaluating lr_L2 ...


/Users/carlagonzalez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/carlagonzalez/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Average training accuracy for model lr_L2 = 0.6359237536656892
Average validation accuracy for model lr_L2 = 0.6228739002932551
-----------------------------------
Evaluating svm_linear ...
Average training accuracy for model svm_linear = 0.6309384164222874
Average validation accuracy for model svm_linear = 0.6222873900293255
-----------------------------------
Evaluating svm_poly ...
Average training accuracy for model svm_poly = 0.6464809384164222
Average validation accuracy for model svm_poly = 0.6170087976539589
-----------------------------------
Best model for the task is lr_L2 which offers the validation accuracy of 0.6228739002932551


In [123]:
# Storing
import pickle

file_to_write = open("best_model.saved","wb")
pickle.dump(best_model,file_to_write)
file_to_write.close()

file_to_write = open("test_data.dat","wb")
pickle.dump([x_test,y_test],file_to_write)
file_to_write.close()

In [140]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

model_file = open("best_model.saved","rb")
model = pickle.load(model_file)
model_file.close()

test_data_file = open("test_data.dat","rb")
x_test,y_test = pickle.load(test_data_file)
test_data_file.close()

# Test accuracy on our test dataset
y_pred_test = model.predict(x_test)

accuracy = accuracy_score(y_pred_test, y_test.flatten())


print (f"Accuracy on test data {accuracy}")

Accuracy on test data 0.6052631578947368


In [126]:
best_model = LogisticRegression(penalty="l2")

feature_names = features_transformed_df.columns

accuracy_drop_log = {"No ablation":0}

for i in range(len(feature_names)):
    feature_name = feature_names[i]
    print (f"Removing feature {feature_name}")
    
    x_ablated = numpy.delete(_x,i,axis=1) # axis = 1 means columns
    
    cv_scores = cross_val_score(best_model,x_ablated,_y.flatten(), cv=k, n_jobs=4)
    average_cv_score = cv_scores.mean()
    print (f"Mean cross validation accuracy = {average_cv_score}")
    accuracy_drop_log[feature_name] = best_model_valid_accuracy-average_cv_score

Removing feature Unnamed: 0
Mean cross validation accuracy = 0.6299120234604105
Removing feature Battery Voltage AVG
Mean cross validation accuracy = 0.619941348973607
Removing feature Temperature AVG
Mean cross validation accuracy = 0.6217008797653959
Removing feature Relative Humidity AVG
Mean cross validation accuracy = 0.6287390029325512
Removing feature Wind Speed Daily AVG
Mean cross validation accuracy = 0.6222873900293255
Removing feature Solare Radiation AVG
Mean cross validation accuracy = 0.5912023460410557
Removing feature 0
Mean cross validation accuracy = 0.6217008797653959
Removing feature 1
Mean cross validation accuracy = 0.6211143695014663
Removing feature 2
Mean cross validation accuracy = 0.6222873900293255
Removing feature 3
Mean cross validation accuracy = 0.6217008797653959
Removing feature 4
Mean cross validation accuracy = 0.619941348973607
Removing feature 5
Mean cross validation accuracy = 0.6217008797653959
Removing feature 6
Mean cross validation accuracy =

In [128]:
def criteria(l):
    return l[1]

sorted_accs =  sorted(accuracy_drop_log.items(),key=criteria, reverse=True)

print (f"Features are ranked from best to worst (based on how removing them impacts the accuracy of {best_model_name})")
print (f"**************************************")

i=1
for entry in sorted_accs:
    feature_name = entry[0]
    acc_drop = entry[1]
    
    # We do not want to print "No ablation"
    if feature_name != "No ablation":
        print (f"Feature {i}.{feature_name}, drop in acc {acc_drop}")
        i=i+1

Features are ranked from best to worst (based on how removing them impacts the accuracy of lr_L2)
**************************************
Feature 1.Solare Radiation AVG, drop in acc -0.5912023460410557
Feature 2.10, drop in acc -0.6181818181818182
Feature 3.21, drop in acc -0.6187683284457478
Feature 4.7, drop in acc -0.6193548387096774
Feature 5.Battery Voltage AVG, drop in acc -0.619941348973607
Feature 6.4, drop in acc -0.619941348973607
Feature 7.11, drop in acc -0.619941348973607
Feature 8.9, drop in acc -0.6205278592375366
Feature 9.1, drop in acc -0.6211143695014663
Feature 10.18, drop in acc -0.6211143695014663
Feature 11.Temperature AVG, drop in acc -0.6217008797653959
Feature 12.0, drop in acc -0.6217008797653959
Feature 13.3, drop in acc -0.6217008797653959
Feature 14.5, drop in acc -0.6217008797653959
Feature 15.6, drop in acc -0.6217008797653959
Feature 16.8, drop in acc -0.6217008797653959
Feature 17.12, drop in acc -0.6217008797653959
Feature 18.13, drop in acc -0.6217008

With the best model which was logistic regression the feature with the most impact would be Battery voltage and the one with the least unnamed.

Creating a model eliminating features like unnamed, relative humidity and month would help us have a more accurate model.